# Uplift modeling

In [56]:
import sklift

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from xgboost import XGBClassifier

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel

Let's suppose we as a comapny find an action to prevent churn. I will simulate how customers will respond, what kind of profits will come with this and how churn will change.
- Firstly i will fabricate dataset of customers after campaing
- Then i will uplift model 

### Dataset creation

Uderlying assumptions:
- there is a decrease in customer service callls
- There is bigger agrement renewal

In [11]:
data = pd.read_csv('data/telecom_churn.csv')

#### Cust service calls

Let's assume that our campaing decresed number of calls by 15 percent. 

In [12]:
data.CustServCalls.mean() , data.CustServCalls.mean() * 0.85 # mean calls is 1.5, 85% of that is 1.3 

(1.5628562856285628, 1.3284278427842784)

In [15]:
decresd_calls = data.CustServCalls *0.85 # decrease calls by one
decresd_calls.mean()

1.3284278427842784

### Renewal

In [41]:
contract_ren_1 = data[data['ContractRenewal'] == 1]
contract_ren_0 = data[data['ContractRenewal'] == 0]

In [47]:
3333 * 0.92 , 3333 - 3066 

(3066.36, 267)

In [ ]:
s = 3333 # sample size

ratio = 0.92 # 92% renewal rate
ren =3066 # renewed customers
not_ren = 3333 - 3066 # non-renewed customers

ren_sample = contract_ren_1.sample(n =ren, random_state=42, replace = True) # sample 3066 customers from the renewed customers
not_ren_sample = contract_ren_0.sample(n =not_ren, random_state=42, replace = True) # sample 3066 customers from the non-renewed customers

print(ren_sample.shape, not_ren_sample.shape)

fabricated = pd.concat([ren_sample, not_ren_sample], axis=0) # concat the two samples

(3066, 11) (267, 11)


In [53]:
fabricated.head()

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
958,0,91,1,1,3.32,3,232.4,97,89.2,9.30,12.3
1436,0,83,1,1,2.92,1,226.4,117,88.2,11.74,10.8
1256,0,98,1,0,0.00,2,162.8,65,44.0,9.25,6.0
1217,0,80,1,1,2.89,4,166.4,92,77.9,11.92,10.7
1817,0,116,1,0,0.00,2,167.8,119,41.0,7.10,7.3


## Uplift  with increased renewal rate

In [55]:
data.head(2) # original data
fabricated.head(2) # fabricated data after supposed campiang increasing renewal rate

,Churn,AccountWeeks,ContractRenewal,DataPlan,DataUsage,CustServCalls,DayMins,DayCalls,MonthlyCharge,OverageFee,RoamMins
958,0,91,1,1,3.32,3,232.4,97,89.2,9.30,12.3
1436,0,83,1,1,2.92,1,226.4,117,88.2,11.74,10.8
